In [1]:
#!/usr/bin/env python3
import os
import json
import subprocess
import tempfile
from pathlib import Path

def analyze_video(video_path):
    """Analyze video file to get codec, resolution, etc."""
    cmd = [
        'ffprobe',
        '-v', 'error',
        '-select_streams', 'v:0',
        '-show_entries', 'stream=codec_name,width,height,r_frame_rate,pix_fmt',
        '-of', 'json',
        video_path
    ]
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        data = json.loads(result.stdout)
        return data.get('streams', [{}])[0]
    except (subprocess.SubprocessError, json.JSONDecodeError, IndexError):
        print(f"Warning: Could not analyze video {video_path}")
        return {}

def check_videos_compatibility(videos):
    """Check if videos can be concatenated directly or need conversion"""
    if not videos:
        return False, {}
    
    # Analyze first video to get reference specs
    _, first_path, _ = videos[0]
    reference = analyze_video(first_path)
    
    if not reference:
        return False, {}
    
    print(f"Reference video specs: {reference}")
    
    all_compatible = True
    for idx, (filename, path, _) in enumerate(videos[1:], 1):
        specs = analyze_video(path)
        if not specs:
            continue
            
        # Compare key properties
        if (specs.get('codec_name') != reference.get('codec_name') or
            specs.get('width') != reference.get('width') or
            specs.get('height') != reference.get('height') or
            specs.get('pix_fmt') != reference.get('pix_fmt')):
            
            print(f"Video {idx} ({filename}) has incompatible specs: {specs}")
            all_compatible = False
    
    return all_compatible, reference

def create_consistent_videos(videos, target_dir, reference_specs):
    """Convert videos to consistent format for concatenation"""
    consistent_videos = []
    
    # Create target directory if it doesn't exist
    os.makedirs(target_dir, exist_ok=True)
    
    for idx, (filename, path, entry) in enumerate(videos):
        # Output file in target directory
        output_path = os.path.join(target_dir, f"temp_{idx:03d}_{filename}")
        
        # Skip conversion for files that already exist
        if os.path.exists(output_path):
            print(f"Using existing converted file for {filename}")
            consistent_videos.append((filename, output_path, entry))
            continue
        
        # Convert to reference specs
        codec = reference_specs.get('codec_name', 'h264')
        width = reference_specs.get('width', 1280)
        height = reference_specs.get('height', 720)
        pix_fmt = reference_specs.get('pix_fmt', 'yuv420p')
        
        cmd = [
            'ffmpeg',
            '-i', path,
            '-c:v', codec,
            '-s', f"{width}x{height}",
            '-pix_fmt', pix_fmt,
            '-movflags', '+faststart',  # For better streaming
            '-y',  # Overwrite output file
            output_path
        ]
        
        print(f"Converting video {idx+1}/{len(videos)}: {filename}")
        subprocess.run(cmd)
        
        if os.path.exists(output_path):
            consistent_videos.append((filename, output_path, entry))
        else:
            print(f"Warning: Failed to convert {filename}")
    
    return consistent_videos

def concat_videos_with_segments(videos, output_video='concatenated_output_segments.mp4'):
    """Concatenate videos by extracting and joining segments (more precise)"""
    temp_dir = tempfile.mkdtemp(prefix="video_concat_")
    segment_files = []
    
    try:
        for idx, (filename, path, entry) in enumerate(videos):
            print(f"Processing segments for video {idx+1}/{len(videos)}: {filename}")
            
            # Get unique segments
            segments = []
            seen_segments = set()
            
            for segment in entry['segments']:
                segment_key = f"{segment['start_frame']}-{segment['end_frame']}-{segment['duration']}-{segment['effect']}"
                if segment_key in seen_segments:
                    continue
                seen_segments.add(segment_key)
                segments.append(segment)
            
            # Extract each segment
            for seg_idx, segment in enumerate(segments):
                start_time = segment['start_frame'] / 30  # Assuming 30fps
                duration = segment['duration']
                
                segment_file = os.path.join(temp_dir, f"v{idx}_seg{seg_idx}.mp4")
                
                cmd = [
                    'ffmpeg',
                    '-i', path,
                    '-ss', str(start_time),
                    '-t', str(duration),
                    '-c', 'copy',
                    '-y',
                    segment_file
                ]
                
                subprocess.run(cmd, stderr=subprocess.DEVNULL)
                
                if os.path.exists(segment_file):
                    segment_files.append(segment_file)
        
        # Create concat file
        concat_file = os.path.join(temp_dir, "segments.txt")
        with open(concat_file, 'w') as f:
            for segment_file in segment_files:
                f.write(f"file '{segment_file}'\n")
        
        # Concatenate segments
        cmd = [
            'ffmpeg',
            '-f', 'concat',
            '-safe', '0',
            '-i', concat_file,
            '-c', 'copy',
            '-y',
            output_video
        ]
        
        print(f"Concatenating {len(segment_files)} segments...")
        subprocess.run(cmd)
        
    finally:
        # Clean up temp files
        for file in segment_files:
            if os.path.exists(file):
                os.remove(file)
        
        if os.path.exists(concat_file):
            os.remove(concat_file)
        
        os.rmdir(temp_dir)
    
    if os.path.exists(output_video):
        file_size = os.path.getsize(output_video) / (1024 * 1024)  # Size in MB
        print(f"Created concatenated video: {output_video} ({file_size:.2f} MB)")
    else:
        print("Failed to create output video!")

def robust_concat_videos(videos, output_video='concatenated_output.mp4'):
    """More robust video concatenation with format checking and conversion if needed"""
    # Check if videos are compatible for direct concatenation
    compatible, reference_specs = check_videos_compatibility(videos)
    
    if compatible:
        print("All videos are compatible for direct concatenation.")
        # Use direct concatenation
        concat_file = "concat_list.txt"
        with open(concat_file, 'w') as f:
            for _, file_path, _ in videos:
                f.write(f"file '{file_path}'\n")
        
        cmd = [
            'ffmpeg',
            '-f', 'concat',
            '-safe', '0',
            '-i', concat_file,
            '-c', 'copy',
            '-y',
            output_video
        ]
        
        print(f"Running ffmpeg command: {' '.join(cmd)}")
        subprocess.run(cmd)
        
        os.remove(concat_file)
    else:
        print("Videos are not directly compatible. Converting to consistent format...")
        # Convert videos to consistent format
        temp_dir = "temp_converted_videos"
        consistent_videos = create_consistent_videos(videos, temp_dir, reference_specs)
        
        # Create concat file for converted videos
        concat_file = "concat_list.txt"
        with open(concat_file, 'w') as f:
            for _, file_path, _ in consistent_videos:
                f.write(f"file '{file_path}'\n")
        
        # Concatenate converted videos
        cmd = [
            'ffmpeg',
            '-f', 'concat',
            '-safe', '0',
            '-i', concat_file,
            '-c', 'copy',
            '-y',
            output_video
        ]
        
        print(f"Running ffmpeg command with converted videos: {' '.join(cmd)}")
        subprocess.run(cmd)
        
        os.remove(concat_file)
    
    # Verify the output
    if os.path.exists(output_video):
        file_size = os.path.getsize(output_video) / (1024 * 1024)  # Size in MB
        print(f"Created concatenated video: {output_video} ({file_size:.2f} MB)")
        return True
    else:
        print("Failed to create output video using standard methods.")
        print("Trying alternative approach with segment extraction...")
        # Try the segment-based approach as fallback
        concat_videos_with_segments(videos, output_video)
        return os.path.exists(output_video)

# NEW FUNCTION: Added JSON log merging functionality
def merge_json_logs(videos, output_json='concatenated_log.json'):
    """Merge JSON log files, adjusting frame numbers and durations"""
    print(f"Starting JSON log merging process...")
    
    merged_data = []
    current_frame_offset = 0
    current_duration_offset = 0
    
    for idx, (filename, _, entry) in enumerate(videos):
        print(f"Processing JSON for video {idx+1}/{len(videos)}: {filename}")
        
        # Create new entry for the merged file
        new_entry = {
            "filename": filename,  # Keep original filename for reference
            "segments": []
        }
        
        # Process segments, removing duplicates
        seen_segments = set()
        
        for segment in entry['segments']:
            # Create a unique key for the segment
            segment_key = f"{segment['start_frame']}-{segment['end_frame']}-{segment['duration']}-{segment['effect']}"
            
            if segment_key in seen_segments:
                continue  # Skip duplicate segments
            
            seen_segments.add(segment_key)
            
            # Create adjusted segment
            adjusted_segment = {
                "start_frame": segment['start_frame'] + current_frame_offset,
                "end_frame": segment['end_frame'] + current_frame_offset,
                "duration": segment['duration'],
                "effect": segment['effect'],
                "original_video": filename  # Add reference to original video
            }
            
            new_entry['segments'].append(adjusted_segment)
        
        merged_data.append(new_entry)
        
        # Update offsets for next video
        # Assuming the last segment's end_frame is the video's total frames
        last_segment = max(entry['segments'], key=lambda s: s['end_frame'])
        current_frame_offset += last_segment['end_frame']
        
        # For duration, calculate the total duration from unique segments
        unique_durations = 0
        unique_segments_dict = {}
        
        # Create a dictionary of unique segments
        for segment in entry['segments']:
            key = (segment['start_frame'], segment['end_frame'], segment['effect'])
            if key not in unique_segments_dict:
                unique_segments_dict[key] = segment['duration']
        
        # Sum up the durations
        total_duration = sum(unique_segments_dict.values())
        current_duration_offset += total_duration
        
        print(f"  Processed {len(seen_segments)} unique segments (removed {len(entry['segments']) - len(seen_segments)} duplicates)")
    
    # Now create a single entry for the concatenated video
    concatenated_entry = {
        "filename": os.path.basename(output_json).replace('.json', '.mp4'),  # Derive filename from JSON
        "segments": []
    }
    
    # Collect all segments from all videos
    all_segments = []
    for entry in merged_data:
        all_segments.extend(entry['segments'])
    
    # Sort segments by start_frame to maintain temporal order
    all_segments.sort(key=lambda s: s['start_frame'])
    
    # Add all segments to the concatenated entry
    concatenated_entry['segments'] = all_segments
    
    # Write the output JSON
    with open(output_json, 'w') as f:
        json.dump([concatenated_entry], f, indent=2)
    
    total_segments = len(all_segments)
    file_size = os.path.getsize(output_json) / (1024 * 1024)  # Size in MB
    
    print(f"Created merged JSON log: {output_json} ({file_size:.2f} MB)")
    print(f"Total segments in merged log: {total_segments}")
    return output_json

def main():
    # Configure paths
    video_dir = 'corrupted_videos_720P'
    json_file = 'video_effects_data.json'
    output_video = 'concatenated_output.mp4'
    output_json = 'concatenated_log.json'
    
    # Read JSON data
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # Find videos
    videos = []
    for entry in data:
        filename = entry['filename']
        file_path = os.path.join(video_dir, filename)
        
        if os.path.exists(file_path):
            videos.append((filename, file_path, entry))
        else:
            print(f"Warning: Video file not found: {file_path}")
    
    if not videos:
        print("No videos found!")
        return
    
    print(f"Found {len(videos)} videos.")
    
    # Concatenate videos
    video_success = robust_concat_videos(videos, output_video)
    
    # Merge JSON logs
    json_success = False
    if video_success:
        json_path = merge_json_logs(videos, output_json)
        json_success = os.path.exists(json_path)
    
    # Report overall success
    if video_success and json_success:
        print("\nSuccess! Both video concatenation and JSON merging completed.")
        print(f"Video output: {output_video}")
        print(f"JSON output: {output_json}")
    elif video_success:
        print("\nPartial success: Video concatenation completed, but JSON merging failed.")
    elif json_success:
        print("\nPartial success: JSON merging completed, but video concatenation failed.")
    else:
        print("\nFailure: Both video concatenation and JSON merging failed.")

if __name__ == "__main__":
    main()

Found 93 videos.
Reference video specs: {'codec_name': 'mpeg4', 'width': 1280, 'height': 720, 'pix_fmt': 'yuv420p', 'r_frame_rate': '30/1'}
Video 1 (VR_720P-0641_crf_10_ss_00_t_20.0.mp4) has incompatible specs: {'codec_name': 'mpeg4', 'width': 1440, 'height': 720, 'pix_fmt': 'yuv420p', 'r_frame_rate': '30/1'}
Video 2 (VerticalVideo_720P-19fa_crf_10_ss_00_t_20.0.mp4) has incompatible specs: {'codec_name': 'mpeg4', 'width': 720, 'height': 1280, 'pix_fmt': 'yuv420p', 'r_frame_rate': '30/1'}
Video 5 (VerticalVideo_720P-2efc_crf_10_ss_00_t_20.0.mp4) has incompatible specs: {'codec_name': 'mpeg4', 'width': 720, 'height': 1280, 'pix_fmt': 'yuv420p', 'r_frame_rate': '30/1'}
Video 6 (VerticalVideo_720P-6580_crf_10_ss_00_t_20.0.mp4) has incompatible specs: {'codec_name': 'mpeg4', 'width': 720, 'height': 1280, 'pix_fmt': 'yuv420p', 'r_frame_rate': '30/1'}
Video 33 (VR_720P-448a_crf_10_ss_00_t_20.0.mp4) has incompatible specs: {'codec_name': 'mpeg4', 'width': 1280, 'height': 640, 'pix_fmt': 'yuv42

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Converting video 2/93: VR_720P-0641_crf_10_ss_00_t_20.0.mp4


[mp4 @ 0x595287677840] Starting second pass: moving the moov atom to the beginning of the file
[out#0/mp4 @ 0x595287676a40] video:8320kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.044217%
frame=  546 fps=0.0 q=31.0 Lsize=    8323kB time=00:00:18.16 bitrate=3753.3kbits/s speed=38.3x    
ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libop

Converting video 3/93: VerticalVideo_720P-19fa_crf_10_ss_00_t_20.0.mp4


...

[mp4 @ 0x5c170f770840] Starting second pass: moving the moov atom to the beginning of the file
[out#0/mp4 @ 0x5c170f76c040] video:6130kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.062620%
frame=  599 fps=0.0 q=31.0 Lsize=    6134kB time=00:00:19.93 bitrate=2521.0kbits/s speed=40.1x    
ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libop

Converting video 93/93: LyricVideo_720P-7f44_crf_10_ss_00_t_20.0.mp4


[mp4 @ 0x6366a5f7ca80] Starting second pass: moving the moov atom to the beginning of the file
[out#0/mp4 @ 0x6366a5f78140] video:6159kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.071752%
frame=  600 fps=0.0 q=24.8 Lsize=    6163kB time=00:00:19.96 bitrate=2528.6kbits/s speed=55.8x    
ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libop

Running ffmpeg command with converted videos: ffmpeg -f concat -safe 0 -i concat_list.txt -c copy -y concatenated_output.mp4
Created concatenated video: concatenated_output.mp4 (841.59 MB)
Starting JSON log merging process...
Processing JSON for video 1/93: Lecture_720P-1f22_crf_10_ss_00_t_20.0.mp4
  Processed 5 unique segments (removed 5 duplicates)
Processing JSON for video 2/93: VR_720P-0641_crf_10_ss_00_t_20.0.mp4
  Processed 7 unique segments (removed 7 duplicates)
Processing JSON for video 3/93: VerticalVideo_720P-19fa_crf_10_ss_00_t_20.0.mp4
  Processed 8 unique segments (removed 8 duplicates)
Processing JSON for video 4/93: Animation_720P-0116_crf_10_ss_00_t_20.0.mp4
  Processed 7 unique segments (removed 7 duplicates)
Processing JSON for video 5/93: Sports_720P-3338_crf_10_ss_00_t_20.0.mp4
  Processed 6 unique segments (removed 6 duplicates)
Processing JSON for video 6/93: VerticalVideo_720P-2efc_crf_10_ss_00_t_20.0.mp4
  Processed 1 unique segments (removed 1 duplicates)
Proc

[out#0/mp4 @ 0x6295d70397c0] video:861502kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.033781%
size=  861793kB time=00:30:29.90 bitrate=3858.0kbits/s speed=4.45e+03x    
